Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [3]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter

55d5a1d1: Already exists 
80d00ae9: Already exists 
b3117dca: Already exists 
a19181b2: Already exists 
7b2a5bcc: Already exists 
26a3eca5: Already exists 
6b82cd2c: Already exists 
b6830bb3: Already exists 
2b8ea37b: Already exists 
b8614138: Pulling fs layer 
6be54d14: Pulling fs layer 
43dd0307: Pulling fs layer 
02be290e: Pulling fs layer 
04b5c452: Pulling fs layer 
ff505c52: Pulling fs layer 
8963813f: Pulling fs layer 


ff505c52: Extracting  1.174GB/1.704GBBDownloading  1.326GB/1.704GBDownloading  1.691GB/1.704GB

Digest: sha256:ab0f7a25d6c4afe0d1c33d43246af53ad63d2321804d37677e1187a257a2d4d9[K
Status: Downloaded newer image for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf_test
Digest: sha256:0b93e6a1d4e4cd5e0057cf503fce53b8702d2445252b7837e844e50752d2a369
Status: Image is up to date for ziylregistry.azurecr.io/perf_test:latest


Install the onnxpipeline SDK

In [8]:
import onnxpipeline

# test tensorflow
#tensorflow_pipeline = onnxpipeline.Onnxpip('mnist/model')

# test pytorch
#pytorch_pipeline = onnxpipeline.Onnxpip('pytorch')

# test mxnet fail
#mxnet_pipeline = onnxpipeline.Onnxpip('mxnet')

# cntk 
cntk_pipeline = onnxpipeline.Onnxpip('cntk')

# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [10]:
# test tensorflow
#model = tensorflow_pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pytorch_pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test mxnet
#model = mxnet_pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')

# test cntk
model = cntk_pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')



Selected CPU as the process wide default device.

/usr/local/lib/python3.6/site-packages/cntk/cntk_py_init.py:56: UserWarning: Unsupported Linux distribution (sion_id="9"-tps://www.debian.org/). CNTK supports Ubuntu 16.04 and above, only.

  warnings.warn('Unsupported Linux distribution (%s-%s). CNTK supports Ubuntu 16.04 and above, only.' % (__my_distro__, __my_distro_ver__))

/usr/local/lib/python3.6/site-packages/cntk/cntk_py_init.py:90: UserWarning: 



################################################ Missing optional dependency (    MKL     ) ################################################

   CNTK may crash if the component that depends on those dependencies is loaded.

   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Linux-Python#mkl for more information.

############################################################################################################################################



  warnings.warn(WARNING_MSG % ('    MKL     ', 'https://docs.mic

## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs: string

    Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs: string

    Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string

    Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple

    Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [11]:
result = pipeline.perf_test(model=model)
# result = pipeline.perf_test()   # is ok, too

Cores:  6



/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32746 ; hostname=6267fa0a0386 ; expr=cudaSetDevice(device_id_); 

Stacktrace:



Stacktrace:







Setting thread pool size to 0





/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, con



Setting thread pool size to 0

Total time cost:0.447264

Total iterations:10

Average time cost:44.7264 ms



ngraph 44.7264



/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32563 ; hostname=6267fa0a0386 ; expr=cudaSetDevice(device_id_); 

Stacktrace:



Stacktrace:







Setting thread pool size to 0





/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxr







Setting thread pool size to 0

Total time cost:0.0110163

Total iterations:10

Average time cost:1.10163 ms



mkldnn_openmp OMP_NUM_THREADS=1, active 1.10163





Setting thread pool size to 0

Total time cost:0.0192224

Total iterations:10

Average time cost:1.92224 ms







Setting thread pool size to 0

Total time cost:0.0188165

Total iterations:10

Average time cost:1.88165 ms



mkldnn_openmp passive 1.88165





Setting thread pool size to 0

Total time cost:3.67189

Total iterations:10

Average time cost:367.189 ms







Setting thread pool size to 0

Total time cost:0.00779203

Total iterations:10

Average time cost:0.779203 ms



mkldnn_openmp active 0.779203





Setting thread pool size to 0

Total time cost:1.34124

Total iterations:10

Average time cost:134.124 ms







Setting thread pool size to 0

Total time cost:1.32105

Total iterations:10

Average time cost:132.105 ms



cpu 132.105





Setting thread pool size to 0

Total time cost:3.46151

Total iterati

# Run parameters

(1) convert-to-onnx-output: string

    Required. The path of the model that wants to be performed.
    
(2) output-perf-result-path: string

    Required. The path of the result.
    

In [5]:
pipeline.print_result(result)
#pipeline.print_result() # is ok, too

mkldnn_openmp active 0.589886 ms

cpu 0.59804 ms

mkldnn 0.67346 ms

cpu_openmp active 0.733781 ms

mkldnn_openmp OMP_NUM_THREADS=6, active 0.788804 ms

cpu_openmp OMP_NUM_THREADS=6, active 0.83225 ms

cpu_openmp OMP_NUM_THREADS=5, active 0.842299 ms

mkldnn_openmp OMP_NUM_THREADS=5, active 0.857851 ms

mkldnn_openmp OMP_NUM_THREADS=4, active 0.865186 ms

mkldnn_openmp OMP_NUM_THREADS=3, active 0.912573 ms

cpu_openmp OMP_NUM_THREADS=4, active 0.93001 ms

cpu_openmp OMP_NUM_THREADS=6, passive 0.936398 ms

cpu_openmp OMP_NUM_THREADS=5, passive 0.976594 ms

cpu_openmp OMP_NUM_THREADS=3, active 0.997364 ms

cpu_openmp OMP_NUM_THREADS=4, passive 1.01441 ms

cpu_openmp OMP_NUM_THREADS=2, active 1.01537 ms

mkldnn_openmp OMP_NUM_THREADS=2, active 1.05105 ms

cpu_openmp OMP_NUM_THREADS=3, passive 1.08854 ms

cpu_openmp OMP_NUM_THREADS=2, passive 1.09684 ms

mkldnn_openmp OMP_NUM_THREADS=1, active 1.12063 ms

mkldnn_openmp OMP_NUM_THREADS=5, passive 1.13045 ms

mkldnn_openmp OMP_NUM_THREADS=4,

# netron

In [5]:
# still trying
import netron
netron.start('../deepthink.onnx')
from IPython.display import IFrame
IFrame('http://10.161.18.106:8080', width=700, height=350)

Serving '../deepthink.onnx' at http://localhost:8080
